# Process External Data

Proccess External CO2/CH4/Ar/water isotope data.

Sourced from preprints:
- https://www.researchgate.net/publication/387341317_Ice_cores_from_the_Allan_Hills_Antarctica_show_relatively_stable_atmospheric_CO2_and_CH4_levels_over_the_last_3_million_years
- https://www.researchgate.net/publication/387779202_Global_ocean_heat_content_over_the_past_3_million_years



In [101]:
# import packages
import pandas as pd
import numpy as np


In [102]:
path_to_data = '../../data/broader_context/'

In [103]:
# load data

mot = pd.read_excel(path_to_data + 'raw_preprint/mot.xlsx')
mot.head()

,Core,top depth,bottom depth,Snapshot,d4038Ar,d4038 err,d3836Ar,d3836 err,d40Ar atm,d40Ar atm err,...,MOT err,MOT abs err,Unnamed: 25,Benthic T component,Benthic T err,Benthic T abs err,Unnamed: 29,T,dc,dw
0,ALHIC1901,134.465,134.525,'short',0.106849,0.005395,0.161073,0.004772,-0.056616,0.004081,...,0.286928,0.576478,NaN,3.567432,0.078090,0.156559,NaN,1.59149,3.56,0.0
1,ALHIC1901,135.195,135.355,'short',0.123546,0.005395,0.173132,0.004772,-0.051560,0.004081,...,0.337964,0.603506,NaN,3.661792,0.092419,0.164707,NaN,NaN,NaN,NaN
2,ALHIC1901,136.185,136.315,'short',0.101380,0.005395,0.154252,0.004772,-0.051231,0.004081,...,0.229267,0.550057,NaN,3.428325,0.061955,0.148297,NaN,NaN,NaN,NaN
3,ALHIC1901,136.575,136.665,'short',0.126299,0.005395,0.174327,0.004772,-0.051566,0.004081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALHIC1901,136.575,136.665,'short',0.111874,0.005395,0.155605,0.004772,-0.041526,0.004081,...,0.210939,0.542674,NaN,3.339287,0.056737,0.145618,NaN,NaN,NaN,NaN


In [104]:
ghg = pd.read_excel(path_to_data + 'raw_preprint/ghg.xlsx', sheet_name='Data')
ghg.head()

,Ice Core,Top Depth,Age,CO2 single,CO2 replicate,CO2 replicate.1,CO2 mean,CH4 single,CH4 replicate,CH4 replicate.1,...,Excluded from pristine dataset?,CO2 from δ13C method single,CO2 from δ13C method replicate,CO2 from δ13C method mean,δ13C single,δ13C replicate,δ13C mean,Corrected CO2 mean,Corrected CO2 high,Corrected CO2 low
0,NaN,m,ka,ppm,ppm,ppm,ppm,ppb,ppb,ppb,...,NaN,ppm,ppm,permil,permil,permil,permil,ppm,ppm,ppm
1,ALHIC1901,121.925,456.034516,230.516125,NaN,NaN,230.516125,467.925575,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALHIC1901,123.3,558.462754,224.679373,NaN,NaN,224.679373,461.126615,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALHIC1901,124.88,826.535983,271.637591,NaN,NaN,271.637591,526.984105,NaN,NaN,...,YES,273.573064,NaN,273.573064,-7.922,NaN,-7.922,253.8728,260.136435,247.324455
4,ALHIC1901,130.655,677.639458,293.454916,NaN,NaN,293.454916,527.458366,NaN,NaN,...,YES,293.158174,NaN,293.158174,-8.071,NaN,-8.071,270.681733,277.449522,263.606318


In [105]:
# filter for  data where the 'Core' colum is ALHIC1901
ghg = ghg[ghg['Ice Core'] == 'ALHIC1901']
mot = mot[mot['Core'] == 'ALHIC1901']

In [106]:
# make a new colum - depth
ghg['top_depth'] = ghg['Top Depth']
mot['top_depth'] = mot['top depth ']#(mot['top depth ']+mot['bottom depth '])/2

ghg['co2']  = ghg['CO2 mean']
ghg['ch4']  = ghg['CH4 mean']
ghg['tac']  = ghg['TAC mean']
ghg['excluded'] = ghg['Excluded from pristine dataset?']
mot['age'] = mot['Argon Age']
mot['age_err'] = mot['Age err']
mot['age_abs_err'] = mot['Age absolute err']

# if excluded is 'yes' then set to 1, else 0
ghg['excluded'] = np.where(ghg['excluded'] == 'YES', 1, 0)

In [107]:
# pull out the key columns
ghg_col = ['top_depth', 'co2','ch4','tac','excluded']
mot_col = ['top_depth','age','age_err','age_abs_err']

# combine the dataframes
df = pd.merge(mot[mot_col], ghg[ghg_col], on='top_depth', how='outer')
df.head()

,top_depth,age,age_err,age_abs_err,co2,ch4,tac,excluded
0,134.465,857.814428,61.828992,112.812038,235.658149,455.129614,0.081413,0.0
1,135.195,781.213014,61.828992,105.864909,249.427331,514.208784,0.067832,0.0
2,136.185,776.228876,61.828992,105.420361,250.962385,492.169663,0.078919,0.0
3,136.575,781.301937,61.828992,105.872849,254.145065,480.798114,0.084161,0.0
4,136.575,629.176721,61.828992,92.805014,254.145065,480.798114,0.084161,0.0


In [108]:
# add matching d18O
argon_full = pd.read_excel(path_to_data + 'raw_GLOBUS/ALHIC1901_ArAges_Princeton_PRELIM_Aug24.xlsx', sheet_name='ALHIC1901')
ghg_full = pd.read_excel(path_to_data + 'raw_GLOBUS/ALHIC1901_AirSummary_OSU_PRELIM_2023.xlsx',sheet_name='Summary (means)')

In [109]:
for _, row in argon_full.iterrows():
    matching_index = df[df['top_depth'] == row['top depth']].index
    if not matching_index.empty:
        df.loc[matching_index, 'd18O_Ar'] = row['del18OSMOW']

for _, row in ghg_full.iterrows():
    matching_index = df[df['top_depth'] == row['Top Depth']].index
    if not matching_index.empty:
        df.loc[matching_index, 'd18O_ch4'] = row['d18Oice (from CH4)']
        df.loc[matching_index, 'd18O_co2'] = row['d18Oice (from CO2)']
        df.loc[matching_index, 'd18O_Ar'] = row['d18Oice (from Ar)']
        #df.loc[matching_index, 'dxs_ch4'] = row['dexcess (from CH4)']
        #df.loc[matching_index, 'dxs_co2'] = row['dexcess (From CO2)']
        #df.loc[matching_index, 'dxs_Ar'] = row['dexcess (from Ar)']

# make a column which is the nan mean of the d18O_Ar, d18O_ch4, and d18O_co2 columns
df['d18O'] = df[['d18O_Ar', 'd18O_ch4', 'd18O_co2']].mean(axis=1)
#df['dxs'] = df[[ 'dxs_ch4', 'dxs_co2','d18O_Ar]].mean(axis=1)

In [110]:
# let's add in the bottom 5m of water iso
# load the data
water_iso = pd.read_excel(path_to_data + 'raw_GLOBUS/ALHIC1901_dD-basal5m_UW_PRELIM_UNKNOWN.xlsx',sheet_name='data')
water_iso['top_depth'] = water_iso['depth_appx']
water_iso['d18O'] = water_iso['d18O_vsmow']
water_iso['dxs'] = water_iso['dxs_vsmow']
cols = ['top_depth','d18O','dxs']

# remove all rows where the floor of 10*['isolab_ID']  is 2284 or 2304
water_iso = water_iso[~water_iso['isolab_ID'].apply(lambda x: str(x).startswith('228.4') or str(x).startswith('230.4'))]


# add the water iso data to the df
df = pd.concat([df,water_iso[cols]], ignore_index=True)

In [111]:
# let's add water isotope data in the 135-142m range (Lindsey's data)
water_iso = pd.read_excel(path_to_data + 'raw_GLOBUS/ALHIC1901_dD-135-142m_UW_PRELIM_UNKNOWN.xlsx',sheet_name='UW_1')
water_iso['top_depth'] = water_iso['depth_appx']
water_iso['d18O'] = water_iso['d18O_vsmow']
water_iso['dxs'] = water_iso['dxs_vsmow']
cols = ['top_depth','d18O','dxs']

# remove all rows where the floor of 10*['isolab_ID']  is 2284 or 2304
water_iso = water_iso[~water_iso['isolab_ID'].apply(lambda x: str(x).startswith('228.4') or str(x).startswith('230.4'))]


# add the water iso data to the df
df = pd.concat([df,water_iso[cols]], ignore_index=True)


In [112]:
# set any ch4 value >1000 to nan
df.loc[df['ch4'] > 1000, 'ch4'] = np.nan


In [113]:
# let's add dust data
dust = pd.read_excel(path_to_data + 'raw_GLOBUS/ALHIC1901_dust_UCSD_PRELIM_11OCT23.xlsx', sheet_name='final comparison')
dust['top_depth'] = dust['TD']
conc_cols = ['ppb1to5','ppb1to10','ppb1to15','ppb1to20','ppb1to25','ppb1to30']
dust['concentration'] = dust[conc_cols].sum(axis=1)

# add the water iso data to the df
cols = ['top_depth','concentration']
df = pd.concat([df,dust[cols]], ignore_index=True)


In [114]:
# save as a csv
df.to_csv(path_to_data + 'broader_context_clean.csv', index=False)

In [115]:
water_iso['isolab_ID']

0       1.201
1       1.202
2       1.203
3       1.204
4       1.205
        ...  
250    10.230
251    10.240
252    10.250
253    10.260
254    10.270
Name: isolab_ID, Length: 255, dtype: float64

In [116]:
water_iso

,sample_ID,isolab_ID,depth_appx,dD_vsmow,dD_RMSE,d18O_vsmow,d18O_RMSE,dxs_vsmow,dxs_RMSE,top_depth,d18O,dxs
0,ALHIC1901_201-2,1.201,135.51,-294.52,0.5,-36.87,0.1,0.41,1,135.51,-36.87,0.41
1,ALHIC1901_201-2,1.202,135.52,-292.11,0.5,-36.62,0.1,0.87,1,135.52,-36.62,0.87
2,ALHIC1901_201-2,1.203,135.53,-291.47,0.5,-36.60,0.1,1.34,1,135.53,-36.60,1.34
3,ALHIC1901_201-2,1.204,135.54,-292.84,0.5,-36.72,0.1,0.94,1,135.54,-36.72,0.94
4,ALHIC1901_201-2,1.205,135.55,-296.16,0.5,-37.15,0.1,1.02,1,135.55,-37.15,1.02
...,...,...,...,...,...,...,...,...,...,...,...,...
250,ALHIC1901_210-2,10.230,141.47,-296.44,0.5,-37.28,0.1,1.80,1,141.47,-37.28,1.80
251,ALHIC1901_210-2,10.240,141.48,-299.28,0.5,-37.59,0.1,1.47,1,141.48,-37.59,1.47
252,ALHIC1901_210-2,10.250,141.49,-303.18,0.5,-38.02,0.1,0.95,1,141.49,-38.02,0.95
253,ALHIC1901_210-2,10.260,141.50,-306.58,0.5,-38.40,0.1,0.64,1,141.50,-38.40,0.64
